说明：Inception Moudel

1、卷积核超参数选择困难，自动找到卷积的最佳组合。

2、1x1卷积核，不同通道的信息融合。使用1x1卷积核虽然参数量增加了，但是能够显著的降低计算量(operations)

3、Inception Moudel由4个分支组成，要分清哪些是在Init里定义，哪些是在forward里调用。4个分支在dim=1(channels)上进行concatenate。24+16+24+24 = 88

4、GoogleNet的Inception(Pytorch实现):https://www.cnblogs.com/Mrzhang3389/p/10127157.html

---
代码说明：  
1、先使用类对Inception Moudel进行封装

2、先是1个卷积层(conv,maxpooling,relu)，然后inceptionA模块(输出的channels是24+16+24+24=88)，接下来又是一个卷积层(conv,mp,relu),然后inceptionA模块，最后一个全连接层(fc)。

3、1408这个数据可以通过x = x.view(in_size, -1)后调用x.shape得到。

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差--->(1,28,28)
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
 
# design model using class
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        # 链路1 in_channels --> out_channels(16),1*1卷积核,不需要padding就能保持图像宽和高不变
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        
        # 链路2
        #in_channels --> out_channels(16),1*1卷积核，不需要padding就能保持图像宽和高不变
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        #in_channels（16） --> out_channels(24),5*5卷积核，padding(只在一个channel内操作)为2（卷积核计算得到5/2=2）是为了保持图像宽度和高度不变
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        # 链路3
        #in_channels --> out_channels(16),1*1卷积核，不需要padding就能保持图像宽和高不变
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        #in_channels（16） --> out_channels(24),3*3卷积核，padding(只在一个channel内操作)为1（卷积核计算得到3/2=1）是为了保持图像宽度和高度不变
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        #in_channels（24） --> out_channels(24),3*3卷积核，padding(只在一个channel内操作)为1（卷积核计算得到3/2=1）是为了保持图像宽度和高度不变
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        # 链路4
        #in_channels --> out_channels(24),1*1卷积核，不需要padding就能保持图像宽和高不变
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
 
    def forward(self, x):
        # 链路1
        branch1x1 = self.branch1x1(x)
        # 链路2
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        # 链路3
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        # 链路4
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        # 4个链路拼接,outputs chneels(24*3+16=88)，拼接的前提是4个链路输出的维度中（batch,channel,width,high）中（b,w,h）保持一样，
        #在c的维度上进行拼接
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim=1) # b,c,w,h  c对应的是dim=1
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5) # 88 = 24x3 + 16
 
        self.incep1 = InceptionA(in_channels=10) # 与conv1 中的10对应
        self.incep2 = InceptionA(in_channels=20) # 与conv2 中的20对应
 
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10) 
 
 
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))#
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
 
        return x
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        ## forward
        # 获得模型预测结果
        outputs = model(inputs)
        # 交叉熵代价函数
        loss = criterion(outputs, target)
        # backward
        loss.backward()
        # update
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,   300] loss: 0.949
[1,   600] loss: 0.202
[1,   900] loss: 0.150
accuracy on test set: 96 % 
[2,   300] loss: 0.119
[2,   600] loss: 0.106
[2,   900] loss: 0.094
accuracy on test set: 97 % 
[3,   300] loss: 0.085
[3,   600] loss: 0.082
[3,   900] loss: 0.077
accuracy on test set: 98 % 
[4,   300] loss: 0.069
[4,   600] loss: 0.071
[4,   900] loss: 0.065
accuracy on test set: 98 % 
[5,   300] loss: 0.063
[5,   600] loss: 0.061
[5,   900] loss: 0.054
accuracy on test set: 98 % 
[6,   300] loss: 0.051
[6,   600] loss: 0.054
[6,   900] loss: 0.049
accuracy on test set: 98 % 
[7,   300] loss: 0.047
[7,   600] loss: 0.046
[7,   900] loss: 0.046
accuracy on test set: 98 % 
[8,   300] loss: 0.043
[8,   600] loss: 0.039
[8,   900] loss: 0.046
accuracy on test set: 98 % 
[9,   300] loss: 0.041
[9,   600] loss: 0.039
[9,   900] loss: 0.039
accuracy on test set: 98 % 
[10,   300] loss: 0.033
[10,   600] loss: 0.039
[10,   900] loss: 0.038
accuracy on test set: 98 % 


---
说明：   

1、要解决的问题：梯度消失

2、跳连接，H(x) = F(x) + x,张量维度必须一样，加完后再激活。不要做pooling，张量的维度会发生变化。

---
代码说明：

1、先是1个卷积层(conv,maxpooling,relu)，然后ResidualBlock模块，接下来又是一个卷积层(conv,mp,relu),然后esidualBlock模块模块，最后一个全连接层(fc)。

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
 
# design model using class
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.channels = channels
        #ResidualBlock的 输入输出channel 一样才能最后和输入相加
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)#图像宽和高保持不变
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
 
    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x + y)
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5) # 88 = 24x3 + 16
 
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
 
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(512, 10) # 暂时不知道512咋能自动出来的
 
 
    def forward(self, x):
        in_size = x.size(0)
 
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
 
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
 
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.529
[1,   600] loss: 0.159
[1,   900] loss: 0.114
accuracy on test set: 97 % 
[2,   300] loss: 0.094
[2,   600] loss: 0.081
[2,   900] loss: 0.071
accuracy on test set: 98 % 
[3,   300] loss: 0.061
[3,   600] loss: 0.061
[3,   900] loss: 0.059
accuracy on test set: 98 % 
[4,   300] loss: 0.047
[4,   600] loss: 0.054
[4,   900] loss: 0.048
accuracy on test set: 98 % 
[5,   300] loss: 0.041
[5,   600] loss: 0.042
[5,   900] loss: 0.040
accuracy on test set: 98 % 
[6,   300] loss: 0.037
[6,   600] loss: 0.036
[6,   900] loss: 0.036
accuracy on test set: 98 % 
[7,   300] loss: 0.031
[7,   600] loss: 0.032
[7,   900] loss: 0.032
accuracy on test set: 98 % 
[8,   300] loss: 0.027
[8,   600] loss: 0.030
[8,   900] loss: 0.027
accuracy on test set: 98 % 
[9,   300] loss: 0.026
[9,   600] loss: 0.025
[9,   900] loss: 0.025
accuracy on test set: 98 % 
[10,   300] loss: 0.022
[10,   600] loss: 0.021
[10,   900] loss: 0.025
accuracy on test set: 98 % 
